In [2]:
%matplotlib inline
import os
import cv2
import numpy as np
import tensorflow as tf

# Load the MPIIGaze model
root_mpi_dir = os.path.join('')
model = tf.keras.models.load_model('C:/MPIIGaze.tar.gz')

# Load the face and eye cascades for detecting faces and eyes
#face_cascade = cv2.CascadeClassifier('path/to/haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('path/to/haarcascade_eye.xml')

# Open the camera for capturing frames
cap = cv2.VideoCapture(0)

while True:
    # Capture a frame from the camera
    ret, frame = cap.read()
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the grayscale image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    # Loop through each detected face
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Extract the region of interest (ROI) corresponding to the face
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes in the ROI
        eyes = eye_cascade.detectMultiScale(roi_gray)
        
        # Loop through each detected eye
        for (ex, ey, ew, eh) in eyes:
            # Draw a rectangle around the eye
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
            
            # Extract the region of interest (ROI) corresponding to the eye
            eye_gray = roi_gray[ey:ey+eh, ex:ex+ew]
            eye_color = roi_color[ey:ey+eh, ex:ex+ew]
            
            # Preprocess the eye image
            eye_resized = cv2.resize(eye_gray, (64, 64))
            eye_resized = eye_resized.astype('float32') / 255.0
            eye_resized = np.expand_dims(eye_resized, axis=-1)
            eye_resized = np.expand_dims(eye_resized, axis=0)
            
            # Predict the gaze direction using the MPIIGaze model
            gaze_direction = model.predict(eye_resized)
            
            # Draw a line indicating the predicted gaze direction
            center_x = ex + ew // 2
            center_y = ey + eh // 2
            length = 50
            dx = length * gaze_direction[0][0]
            dy = -length * gaze_direction[0][1]
            cv2.line(roi_color, (center_x, center_y), (int(center_x + dx), int(center_y + dy)), (0, 0, 255), 2)
    
    # Display the resulting frame
    cv2.imshow('Eye Tracking', frame)
    
    # Wait for a key press to exit the program
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
cap.release()
cv2.destroyAllWindows()


OSError: Unable to open file (file signature not found)